# **Density of states (DOS)**

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/master/notebook/band-theory/density_of_states.ipynb

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Goals**

This notebook demonstrates various approaches for the numerical calculation 
of the density of states (DOS) for a 3D free-electron model in periodic 
boundary conditions.
   
    * Learn the methods to calculate the density of states.
    * Examine the resulting DOS and evaluate the accuracy of various methods to compute the DOS.

## **Background theory**

[More on the background theory.](./theory/theory_density_of_states.ipynb)

<details close>
<summary style="font-size: 20px">Free electron model (3D)</summary>
In the free electron model, the electrons move "freely" without any 
potential ($V=0$). The eigenfunctions of the Schrödinger equation for 
free electrons are (apart for normalization):
    
$$\large \psi(\vec{r}) = e^{i\vec{k} \vec{r}}$$

The dispersion is:

$$\large E = \frac{\hbar^2k^2}{2m}$$

where, $k = k_x + k_y + k_z$. From the dispersion, one can see that the 
energy isosurface is a sphere in the reciprocal space (k-space) as shown in 
the interactive figure. Hence, the number of states for a given wavevector 
$k$ is calculated by constructing a spherical shell of radius $k$ and 
thickness $dk$. The volume of this spherical shell is $4\pi k^2dk$. The 
formula of the DOS can be de derived as:

$$\large D(E) = \frac{V}{2\pi^2}(\frac{2m}{\hbar})^{\frac{3}{2}}\sqrt{E}$$

where V is the total volume. One can see that the DOS is proportional to
$\sqrt{E}$. Please read more at the Wikipedia:
<a href="https://en.wikipedia.org/wiki/Free_electron_model">free electron model</a>
</details>

<details close>
<summary style="font-size: 20px">Density of states (DOS)</summary>
The density of states (DOS) is the density of available electronic states per 
unit volume and energy, which is defined as:

$$\large D(E)=\frac{1}{V}\sum_{n,\vec k}\delta(E-E_{n\vec k})$$

where $V$ is the volume, $\delta$ is a Dirac's delta, $E_{n\vec k} = E_n(\vec k)$ 
is the energy for the n-th band at k-point $\vec k$, and the sum is over all 
band $n$ and all k-vectors $\vec k$.

The simplest approximation to $D(E)$ is obtained by considering a finite number 
of k points on a regular grid, dividing the energy range in small bins, and 
computing a histogram of the energies obtained on the finite k-point grid. 
The resulting histogram is an approximation to the density of states 
(after appropriate normalization). However, the approximation is quite crude 
unless the number of k-points is very large, and the bin size on the energy 
axis is chosen appropriately.

To improve the results, one can "smear" the histogram, e.g. instead of simply
accumulating elements into bins, we can sum Gaussian functions centered at 
the energy $E(k_i)$, with a fixed standard deviation $\sigma$.
This method is called Gaussian smearing, which makes the DOS curve 
much smoother than a simple histogram already for relatively coarse k-grids. 
However, this method introduces some error when trying to estimate the 
position of band edges from the DOS (with an error of the order of $\sigma$).
Mathematically, the DOS is approximated by the following expression:

$$\large D(E)=\sum_{n,\vec k}\frac{1}{\sigma\sqrt{2\pi}}e^{-\frac{(x-E_{n\vec k})^2}{2\sigma^2}}$$
    
Finally, the third method we describe here is the
linear tetrahedron interpolation (LTI). In this method, the volume in 
reciprocal space is split into small tetrahedra. The energy at each corner 
is computed similarly to the previous case. Typically, a regular grid is chosen 
also in this case, and each small volume - that typically has a shape of a cube or, 
more generally, a parallelepiped - is split into tetrahedra: the image below 
demonstrates how to split a cubic reciprocal space volume into six tetrahedra.

<div class="container" style="text-align: center; width: 500px;">
  <img src="images/LTI.png" alt="Linear tetrahedron interpolation" class="image">
  <div class="overlay">Linear tetrahedron interpolation (image from 
    <a href="http://www.physics.okayama-u.ac.jp/jeschke_homepage/CMSST2016/chapter1.pdf">this PDF file</a>)
  </div>
</div>
    
Then, the method assumes that, within a tetrahedron, the energy behaves 
linearly; therefore a linear interpolation is employed to obtain the value 
of the energy in any point inside the tetrahedron, knowing the values of 
the energy at its fours corners. Thanks to this, it is possible to calculate 
much more accurately the portion of the volume of each tetrahedron that is 
above or below a given energy, making the resulting DOS much more accurate 
than a simple histogram obtained from the value of the energy at its four corners.
</details>

## **Tasks and exercises**

1. Investigate the influence of the number of k-points on the resulting DOS.
    <details>
    <summary>Hints</summary>
    In the right panel, the blue line is the analytical solution for the DOS. 
    By choosing the different number of kpoints from the slider, we can compare 
    the calculated results with the analytical solution. The density of states 
    is a probability distribution. The kpoints sampling is shown as red dots 
    in the left panel. The more kpoints the better results we can obtain. 
    </details>   

2. Which method gives most accurate results? Which method is fastest and why?

    <details>
    <summary>Hints</summary>
    Linear tetrahedra interpolation (LTI) is an accurate numerical method, 
    which interpolates the 3D kpoints grid. LTI method can give much better 
    results rather than a simple histogram. Gaussian smearing makes the 
    histogram plot much smoother, which is closer to the analytical 
    solution. The histogram method is a simple statistic of the eigenvalues, 
    which should be the fastest to compute.
    </details>

3. Why do the calculated results start to get diverge when the energy level is 
higher than a certain value? Could you explain it with the k-space plot?

    <details>
    <summary>Hints</summary>
    In the free electron model, the energy isosurface is a sphere shown in 
    the left panel. The kpoints grid must be larger than the energy 
    isosurface to obtain the correct DOS at the energy level. Here, we 
    have a fixed length of the kpoints grid. When the energy is larger than 
    about 0.31, the kpoints grid cannot include the whole sphere (check it by 
    clicking on the right panel to move the isovalue above and below 0.31). 
    </details>

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Interactive visualization
(be patient, it might take a few seconds to load)

In [ ]:
import numpy as np
import seekpath
import re
import os
import matplotlib
from ase.dft.dos import linear_tetrahedron_integration as lti
from ase.dft.kpoints import monkhorst_pack
from ase.cell import Cell
from scipy.stats import norm
import plotly.graph_objects as go
import plotly.express as px
import time
import matplotlib.pyplot as plt
from ipywidgets import Button, RadioButtons, Layout, IntSlider, HBox, VBox, Checkbox, Label, FloatSlider, Output, HTML
from datetime import datetime

%matplotlib widget

In [ ]:
def get_kernel_id():
    """Get the current kernel ID, to distinguish different users.
    
    Call this only from within python jupyter notebooks.
    """
    from IPython.lib import kernel
    connection_file_path = kernel.get_connection_file()
    connection_file = os.path.basename(connection_file_path)
    kernel_id = connection_file.split('-', 1)[1].split('.')[0]
    return kernel_id

In [ ]:
def log_button(name):
    try:
        allow_datacollection
    except:
        pass
    else:
        if allow_datacollection:
            log_file = open('../log.dat', 'a+');
            log_file.write(datetime.now().strftime("%Y-%m-%d %H:%M:%S") + ' ')
            log_file.write(get_kernel_id() + ' ')
            log_file.write(name + ' ')
            log_file.write(str(nkpt.value) + '\n')
            log_file.close()

In [ ]:
def _compute_dos(kpts, G, ranges):
    """initial all the engienvalue according to the kpoints
    
    Args: 
        kpts: a array of kpts (kx, ky, kz)
        G: the reciprocal lattice vectors (3x3)
        ranges: the range of the reciprocal lattice 
        
    Returns:
        The eigenvalues of the free electron model.
    """
    eigs = []
    n = ranges
    
    for i in range(-n, n+1):
        for j in range(-n, n+1):
            for k in range(-n, n+1):
                g_vector = i*G[0] + j*G[1] + k*G[2]
                eigs.append(np.sum(0.5*(kpts + g_vector)**2, axis=3))

    eigs = np.moveaxis(eigs, 0, -1)
    return eigs
   
def _compute_total_kpts(G, grange=0):
    """Get all the kpoints 
    
    Args:
        G: the reciprocal lattice vectors (3x3)
        grange: the range of the reciprocal lattice
        
    Returns:
        The kpoints (kx, ky, kz) as a array
    """
    tot_kpts = []
    n = grange
    
    shape = (nkpt.value, nkpt.value, nkpt.value)
    kpts = np.dot(monkhorst_pack(shape), G).reshape(shape + (3,))
    kpts = kpts.reshape(nkpt.value**3, 3)

    for i in range(-n, n+1):
        for j in range(-n, n+1):
            for k in range(-n, n+1):
                g_vector = i*G[0] + j*G[1] + k*G[2]
                tot_kpts.extend(kpts+g_vector)
    return np.array(tot_kpts)
    

In [ ]:
alat_bohr = 7.72

lattices = np.zeros((3, 3, 3));

lattices[0] = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) * alat_bohr / 2.0;
lattices[1] = np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]]) * alat_bohr / 2.0;
lattices[2] = np.array([[-1, 1, 1], [1, -1, 1], [1, 1, -1]]) * alat_bohr / 2.0;

#Choose the cubic lattice for using the linear tetrahadron method (ASE)
real_lattice_bohr = lattices[0]

In [ ]:
style = {'description_width': 'initial'}

nkpt = IntSlider(value=4, min=4, max=25, description="Number of kpoints:", style={'description_width': 'initial'}, continuous_update=False)
nbin = IntSlider(value=30, min=30, max=500, description="Number of bins:", layout=Layout(width="300px"), style={'description_width': 'initial'})
gstd = FloatSlider(value=0.01, min=0.01, max=0.1, step=0.01, description="Gaussian $\sigma$ (eV):", layout=Layout(width="300px"), style={'description_width': 'initial'})

#All buttons
btlti = Button(description="Tetrahedra", style = {'button_color':'green'})
bthist = Button(description="Histogram", style = {'button_color':'green'})
btgas = Button(description="Smearing", style = {'button_color':'green'})
btclear = Button(description="Clear plot", style = {'button_color':'green'})

#Ouput for the DOS figure
output = Output()

def compute_dos_lti(c):
    """Compute the DOS uing the ASE linear tetrahedron interpolation method.
    """
    global llti
    
    btlti.disabled = True
    bthist.disabled = True
    btgas.disabled = True
    btclear.disabled = True
    btlti.style = {'button_color':'red'}
    btlti.description = "Running"
    
    log_button('Tetrahedra')
    
    try:
        llti.remove()
    except:
        pass
    
    shape = (nkpt.value, nkpt.value, nkpt.value)
    G = Cell(real_lattice_bohr).reciprocal()*2*np.pi
    kpts = np.dot(monkhorst_pack(shape), G).reshape(shape + (3,))

    eigs = _compute_dos(kpts, G, 0)

    dosx = np.linspace(0, 10, 500)
    dosy = lti(real_lattice_bohr, eigs, dosx)
    
    llti, = ax.plot(dosy, dosx, 'r-', label='LTI')
    ax.legend(loc=1, bbox_to_anchor=(1.3, 1.0))
    
    btlti.disabled = False
    bthist.disabled = False
    btgas.disabled = False
    btclear.disabled = False
    btlti.style = {'button_color':'green'}
    btlti.description="Tetrahedron"

btlti.on_click(compute_dos_lti)

def compute_dos_histogram(c):
    """Compute the DOS as a histogram.
    """
    global lhist
    
    btlti.disabled = True
    bthist.disabled = True
    btgas.disabled = True
    btclear.disabled = True
    bthist.style = {'button_color':'green'}
    
    log_button('Histogram')
    
    try:
        lhist.remove()
    except:
        pass
    
    shape = (nkpt.value, nkpt.value, nkpt.value)
    G = Cell(real_lattice_bohr).reciprocal()*2*np.pi
    kpts = np.dot(monkhorst_pack(shape), G).reshape(shape + (3,))
    eigs = _compute_dos(kpts, G, 0)
    
    hy, hx = np.histogram(eigs.ravel(), bins=nbin.value, range=(0.0, 3.0))
    hy = hy/np.sum(hy*np.diff(hx))*np.shape(eigs)[-1]
    
    lhist = ax.barh(hx[:-1]+np.diff(hx)[0], hy, color='yellow', edgecolor='black', 
                       height=np.diff(hx), label="Histogram")
    ax.legend(loc=1, bbox_to_anchor=(1.3, 1.0))

    btlti.disabled = False
    bthist.disabled = False
    btgas.disabled = False
    btclear.disabled = False
    bthist.style = {'button_color':'green'}
    
bthist.on_click(compute_dos_histogram)

def compute_dos_gaussian(c):
    """Computing the DOS using Gaussian smearing method.
    """
    global lgas
    
    btlti.disabled = True
    bthist.disabled = True
    btgas.disabled = True
    btclear.disabled = True
    btgas.style = {'button_color':'red'}
    btgas.description = "Running"
    
    log_button('Smearing')
    
    try:
        lgas.remove()
    except:
        pass
    
    shape = (nkpt.value, nkpt.value, nkpt.value)
    G = Cell(real_lattice_bohr).reciprocal()*2*np.pi
    kpts = np.dot(monkhorst_pack(shape), G).reshape(shape + (3,))
    eigs = _compute_dos(kpts, G, 0)
    
    gx = np.linspace(-0.03, 5, 500)
    gy = 0.0*gx
    
    for eig in eigs.ravel():
        gy += norm(eig, gstd.value).pdf(gx)
 
    gy = gy/np.size(eigs)*np.shape(eigs)[-1]
    lgas, = ax.plot(gy, gx, 'k--', label="Gaussian smearing")
    ax.legend(loc=1, bbox_to_anchor=(1.3, 1.0))
    
    btlti.disabled = False
    bthist.disabled = False
    btgas.disabled = False
    btclear.disabled = False
    btgas.style = {'button_color':'green'}
    btgas.description = "Smearing"
    
btgas.on_click(compute_dos_gaussian)


def init_dos_plot():
    """Init the DOS plot.
    """
    global hline, ann
    btlti.disabled = True
    bthist.disabled = True
    btgas.disabled = True
    
    analy_x = np.linspace(0, 0.5, 500);
    analy_y = 1.0/(2.0*np.pi**2)*2.0**0.5*analy_x**0.5*(alat_bohr / 2.0)**3.0;
    lanaly, = ax.plot(analy_y, analy_x, 'b', label='Analytical solution')
    
    ax.set_ylim([-0.03, 0.5])
    ax.set_xlim([0, analy_y.max() + 3.1])
    ax.legend(loc=1, bbox_to_anchor=(1.3, 1.0))
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right")
    ax.set_ylabel('Density of States (eV)')
    hline = ax.axhline(0.3, color="red")
    ann = ax.annotate(r"$\frac{\hbar^2k^2}{2m}$ isosurf. (click to move)", xy=(0.2, 0.31), fontsize=10)
    fig.tight_layout()
    
    btlti.disabled = False
    bthist.disabled = False
    btgas.disabled = False

def onclick(event):
    """Click to move the isovalue line (red horizontal line) and update the kpoints plot.
    """
    hline.set_ydata(event.ydata)
    figkpts.data[0].isomin = event.ydata
    figkpts.data[0].isomax = event.ydata
    ann.set_position((0.5, event.ydata + 0.01))
        
with output:
    """Set the figure for the DOS
    """
    global fig, ax
    fig, ax = plt.subplots()
    fig.set_size_inches(3.2, 5.0)
    fig.canvas.header_visible = False
    fig.canvas.layout.width = "380px"
    fig.tight_layout()
    init_dos_plot()
    cid = fig.canvas.mpl_connect('button_press_event', onclick)
    plt.show()
    
def clear_plot(c):
    """Clear the DOS calculated results when the "Clear" button is clicked.
    """
    ax.clear()
    init_dos_plot()
    
btclear.on_click(clear_plot)

In [ ]:
df = px.data.gapminder()

X, Y, Z = np.mgrid[-2:2:40j, -2:2:40j, -2:2:40j]

# Fermi surface
values = 0.5*(X * X + Y * Y + Z * Z)

G = Cell(real_lattice_bohr).reciprocal()*2*np.pi
kpts = _compute_total_kpts(G)

#Init the kpoints plot with the plotly package
figkpts = go.FigureWidget(data=[go.Isosurface(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=values.flatten(),
    opacity=0.5,
    isomin=0.3,
    isomax=0.3,
    surface_count=1,
    caps=dict(x_show=False, y_show=False)),
    go.Scatter3d(x=kpts[:,0], y=kpts[:,1], z=kpts[:,2], mode='markers',
                marker=dict(size=1.5, color='red'))],
    layout=go.Layout(width=450, title='Kpoints (red dots) in reciprocal space and'
                     +'<br>energy isosurface (isovalue can be set by <br> clicking on the left figure)',
                     scene=dict(bgcolor = 'rgb(20, 24, 54)',
                                xaxis = dict(title=r'kx', titlefont_color='white'),
                                yaxis = dict(title=r'ky', titlefont_color='white'),
                                zaxis = dict(title=r'kz', titlefont_color='white'))))


def update_kpts_fig(c):
    """Update the kpoints plot when tuning the kpoints slider.
    """
    kpts = _compute_total_kpts(G)
    
    with figkpts.batch_update():
        figkpts.data[1].x = kpts[:, 0]
        figkpts.data[1].y = kpts[:, 1]
        figkpts.data[1].z = kpts[:, 2]
        
        if nkpt.value >= 8:
            figkpts.data[1].marker['size'] = 1.0
        else:
            figkpts.data[1].marker['size'] = 1.5

def half_sphere():
    """Only show half of the isosurface.
    """
    X, Y, Z = np.mgrid[-6:6:40j, 0:6:40j, -6:6:40j]
    values = 0.5*(X * X + Y * Y + Z * Z)
    figkpts.data[0].x = X.flatten()
    figkpts.data[0].y = Y.flatten()
    figkpts.data[0].z = Z.flatten()
    figkpts.data[0].value = values.flatten()
    

nkpt.observe(update_kpts_fig, names="value")

In [ ]:
#Group buttons with descriptions as labels
method1 = VBox([HBox([bthist, nbin]), Label(value="(Simple histogram of the eigenvalues)")])
method2 = VBox([HBox([btgas, gstd]), Label(value="(Gaussian smearing method)")])
method3 = HBox([btlti, Label(value="(Linear tetrahedron interpolation method)")])
method4 = HBox([btclear, Label(value="(Clear the calculated results)")])

label1 = HTML(value = f"<b><font color='red'>Choose a method to calculate the DOS:</b>")

display(HBox([VBox([figkpts, nkpt, label1, method1, method2, method3]), VBox([output, method4])]))

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

# Legend

(How to use the interactive visualization)

## Interactive figures

The left panel shows the kpoints (red dots) in the reciprocal space (k-space).
A transparent sphere is employed to represent the energy isosurface 
(Fermi surface). The value of the isosurface can be set by clicking on the 
right panel (red horizontal line). Choose the number of kpoints in each 
dimension from the kpoints slider. The left panel will update accordingly 
when the number of the kpoints is changed.

## Controls

Three buttons allow to compute the DOS with the three methods discussed 
earlier. The DOS results will appear in the figure on the right.
Get results with many kpoints might take several seconds.
For the Gaussian smearing method, you can also tune the standard 
deviation $\sigma$ of Gaussian functions with the slider next to 
the "Smearing" button.